In [295]:
# 删除无效数据，去重，更改部分列名称

import pandas as pd
data = pd.read_csv("data.csv", encoding="gbk")
data.rename(columns={'1、姓名（不是 ID）': '姓名'}, inplace=True)
data.rename(columns={'2、ID': 'ID'}, inplace=True)
data.rename(columns={'3、手机号': '手机号'}, inplace=True)
data = data[data['姓名'] != 'aaa']
data = data.drop_duplicates(subset=['姓名', 'ID', '手机号'], keep='last', ignore_index=True)
# print(data)

In [296]:
# 检查是否有疑似相同的问卷

# print(data)
for i in range(len(data)):
	for j in range(i):
		if data.loc[j, '姓名'] == data.loc[i, '姓名']\
		or data.loc[j, 'ID'] == data.loc[i, 'ID']\
		or data.loc[j, '手机号'] == data.loc[i, '手机号']:
			print(f'Warning: Row {j} and Row {i} may be the same person.')
			print(data.loc[j, ['姓名', 'ID', '手机号']])
			print(data.loc[i, ['姓名', 'ID', '手机号']])


姓名              丁田
ID              雾屿
手机号    15957231591
Name: 64, dtype: object
姓名              丁田
ID              丁田
手机号    15957231591
Name: 94, dtype: object


In [ ]:
# 将列名称从款式补全为完整商品

cols = data.columns.tolist()
# print(cols)
phone_visited = False
for i in range(0,len(cols)):
	if cols[i] == '手机号':
		phone_visited = True
		continue
	if not phone_visited:
		continue
	if cols[i].split('、')[0] == cols[i-1].split('、')[0]:
		cols[i] = '('.join(cols[i-1].split('(')[:-1])+'('+cols[i].split('(')[-1]
# print(cols)
data.columns = cols
# print(data.columns)

In [ ]:
# 特殊处理

if data.loc[64, '姓名'] == '丁田' and data.loc[64, 'ID'] == '雾屿':
	data.drop(64, axis=0, inplace=True)
	data = data.reset_index(drop=True)

data.loc[data['ID'] == '菜菜', '17、宽大座垫(黑蓝)'] = 0
# print(data.loc[data['ID'] == '菜菜', '17、宽大座垫(黑蓝)'])
data['4、头巾(30)'] = 0

data.to_csv('output.csv', index=False, sep=',', encoding='gbk')

In [ ]:
# 对列统计

col_sums = data.sum(numeric_only=True, skipna=True).astype(int)

col_sums.to_csv('total.csv', index=True, sep=',', encoding='gbk')

In [ ]:
prices = pd.read_csv("prices.csv", encoding="utf-8")

# print(prices["商品"])
cols = data.columns.tolist()

sum = 0

charge_table = pd.DataFrame(columns=['姓名', '金额'])

for i in range(len(data)):
	total_100 = 0
	for col in cols:
		cnt = data.loc[i, col]
		if (prices['商品'] == col).any():
			if not pd.isna(cnt):
				unit_price = prices.loc[prices['商品'] == col, '单价'].iloc[0]
				total_100 += round(unit_price * 100) * int(cnt)
	sum += total_100
	charge_table.loc[len(charge_table)] = [data.loc[i, '姓名'], total_100 / 100]

# print(sum / 100)

charge_table.to_csv('charge.csv', index=True, sep=',', encoding='gbk')

0 序号 2
0 提交答卷时间 2025/3/7 20:19
0 所用时间 208秒
0 来源 微信
0 来源详情 nan
0 来自IP 124.64.22.242(北京-北京)
0 姓名 刘一鸣
0 ID 柏拉图
0 手机号 13592380817
0 4、头巾(03) nan
0 4、头巾(04) nan
0 4、头巾(05) nan
0 4、头巾(07) nan
0 4、头巾(09) nan
0 4、头巾(12) nan
0 4、头巾(17) nan
0 4、头巾(22) nan
0 4、头巾(26) nan
0 4、头巾(27) nan
0 4、头巾(30) 0
0 5、绑腿(双色版（一对）) nan
0 5、绑腿(三色版（一对）) nan
0 6、手机密封袋(藏青) nan
0 6、手机密封袋(白) nan
0 6、手机密封袋(绿) nan
0 6、手机密封袋(黑) nan
0 6、手机密封袋(蓝) nan
0 6、手机密封袋(粉) nan
0 6、手机密封袋(青) nan
0 7、半指手套(红S) nan
0 7、半指手套(蓝S) nan
0 7、半指手套(灰S) nan
0 7、半指手套(绿S) nan
0 7、半指手套(红M) nan
0 7、半指手套(蓝M) nan
0 7、半指手套(灰M) nan
0 7、半指手套(绿M) nan
0 7、半指手套(红L) nan
0 7、半指手套(蓝L) nan
0 7、半指手套(灰L) nan
0 7、半指手套(绿L) nan
0 7、半指手套(红XL) nan
0 7、半指手套(蓝XL) nan
0 7、半指手套(灰XL) nan
0 7、半指手套(绿XL) nan
0 8、春秋季全指手套(黑绿S) 0.0
0 8、春秋季全指手套(黑S) 0.0
0 8、春秋季全指手套(黑绿M) 0.0
0 8、春秋季全指手套(黑M) 0.0
0 8、春秋季全指手套(黑绿L) 0.0
0 8、春秋季全指手套(黑L) 0.0
0 8、春秋季全指手套(黑绿XL) 0.0
0 8、春秋季全指手套(黑XL) 1.0
0 8、春秋季全指手套(黑绿XXL) 0.0
0 8、春秋季全指手套(黑XXL) 0.0
0 9、夏季长指手套(蓝S) nan
0 9、夏季长指手套(黑S) nan
0 9、夏季长指手套(红S) nan
0 9、夏季长